# Properties

In [1]:
"""
Pistonball v4

Environment Properties:
    Actions:                Either
    Agents:	                20
    Parallel API:	        Yes
    Manual Control:	        Yes
    Action Shape:	        (1,)
    Action Values:	        [-1, 1]
    Observation Shape:	    (457, 120, 3)
    Observation Values:	    [0, 255]
    State Shape:            (560, 880, 3)
    State Values:	        (0, 255)
    Average Total Reward:   -91.2
"""

'\nPistonball v4\n\nEnvironment Properties:\n    Actions:                Either\n    Agents:\t                20\n    Parallel API:\t        Yes\n    Manual Control:\t        Yes\n    Action Shape:\t        (1,)\n    Action Values:\t        [-1, 1]\n    Observation Shape:\t    (457, 120, 3)\n    Observation Values:\t    [0, 255]\n    State Shape:            (560, 880, 3)\n    State Values:\t        (0, 255)\n    Average Total Reward:   -91.2\n'

# Install Libraries

In [3]:
pip install pettingzoo

     |████████████████████████████████| 759 kB 3.2 MB/s 
     |████████████████████████████████| 1.6 MB 51.1 MB/s 
  Created wheel for pettingzoo: filename=PettingZoo-1.11.2-py3-none-any.whl size=874919 sha256=e4c26931e0428a18e15671646729c598cc3aecb31749f56253699f79a12e6dc6
  Stored in directory: /root/.cache/pip/wheels/d0/21/af/972ca3514591f6e2001fdc724a34e2c7f363833e2448de3625
  Created wheel for gym: filename=gym-0.19.0-py3-none-any.whl size=1663114 sha256=0d72c4b3163967f9efe1cafd7a81de608868bf473325c192dc2dc52c2cca42ec
  Stored in directory: /root/.cache/pip/wheels/ef/9d/70/8bea53f7edec2fdb4f98d9d64ac9f11aea95dfcb98099d7712
Successfully built pettingzoo gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3


In [4]:
pip install stable-baselines3

     |████████████████████████████████| 161 kB 4.1 MB/s 


In [5]:
pip install supersuit

     |████████████████████████████████| 47.9 MB 12 kB/s 
  Created wheel for supersuit: filename=SuperSuit-3.1.0-py3-none-any.whl size=53149 sha256=7629e3eec7b737d2a18effcca079fbdfdd0fc95044acf6c173c10408e56c3c3a
  Stored in directory: /root/.cache/pip/wheels/1f/7e/99/74f0986616cedb9c0c37dc29391acdc9a2be663a3c84d2c132
Successfully built supersuit
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [8]:
pip install pygame

     |████████████████████████████████| 11.8 MB 3.2 MB/s 


In [11]:
pip install pymunk

     |████████████████████████████████| 980 kB 4.1 MB/s 


# Import Libraries

In [12]:
from pettingzoo.butterfly import pistonball_v4
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3 import PPO
import supersuit as ss

/usr/local/lib/python3.7/dist-packages/supersuit/__init__.py:20: UserWarning: You're using SuperSuit 3.0, released 7/7/21. The entire codebase has been rewritten or refactored as part of this release. While we've tested it thoroughly, please ensure everything you're doing still works properly and report any issues at https://github.com/PettingZoo-Team/SuperSuit. This warning will be removed 2 months after release.
  warnings.warn("You're using SuperSuit 3.0, released 7/7/21. The entire codebase has been rewritten or refactored as part of this release. While we've tested it thoroughly, please ensure everything you're doing still works properly and report any issues at https://github.com/PettingZoo-Team/SuperSuit. This warning will be removed 2 months after release.")


# Define Environment

In [13]:
env = pistonball_v4.parallel_env(n_pistons=20, local_ratio=0, time_penalty=-0.1, continuous=True,
                                random_drop=True, random_rotate=True, ball_mass=0.75, ball_friction=0.3,
                                ball_elasticity=1.5, max_cycles=125)

# Change Shape of Observations

In [14]:
env = ss.color_reduction_v0(env, mode="B")
env = ss.resize_v0(env, x_size=84, y_size=84)

# stack past 3 frames together to see the ball's acceleration clearly
env = ss.frame_stack_v1(env, 3)

# make policy parameter sharing for multi-agent compatibility
env = ss.pettingzoo_env_to_vec_env_v0(env)

# make environment multiple versions at the same time (8 different environments in parallel)
env = ss.concat_vec_envs_v0(env, 8, num_cpus=4, base_class="stable_baselines3")

# Define PPO Model

In [15]:
# define model
model = PPO(CnnPolicy, env, verbose=3, gamma=0.95, n_steps=256, ent_coef=0.0905168,
            learning_rate=0.00062211, vf_coef=0.042202, max_grad_norm=0.9, gae_lambda=0.99,
            n_epochs=5, clip_range=0.3, batch_size=256)

Using cpu device
Wrapping the env in a VecTransposeImage.


# Training

In [16]:
# TRAINING and SAVING
print("\nTraining is Starting ...\n")

model.learn(total_timesteps=2000)
model.save("policy")


Training is Starting ...

------------------------------
| time/              |       |
|    fps             | 399   |
|    iterations      | 1     |
|    time_elapsed    | 102   |
|    total_timesteps | 40960 |
------------------------------


# Evaluating

In [17]:
# EVALUATING
print("\nEvaluation is Starting ...\n")

env = pistonball_v4.env()
env = ss.color_reduction_v0(env, mode="B")
env = ss.resize_v0(env, x_size=84, y_size=84)
env = ss.frame_stack_v1(env, 3)

# load save policy model
model = PPO.load("policy")

# run the model according to the policy with render open mode
env.reset()
for agent in env.agent_iter():
   obs, reward, done, info = env.last()
   act = model.predict(obs, deterministic=True)[0] if not done else None
   env.step(act)
   env.render()


Evaluation is Starting ...



error: ignored